In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
import matplotlib.pyplot as plt
import copy
import time

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
torch.__version__

import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, transform

In [8]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

!unzip '/content/drive/My Drive/Vision/category_imgs.zip'

train_data = pd.read_csv('/content/drive/My Drive/Vision/train/category_train.csv')
test_data = pd.read_csv('/content/drive/My Drive/Vision/test/category_test.csv')

In [28]:
Category = np.array(data.Category.unique())

class Dataset_Category(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.category = Category

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        path = "/content/category_imgs/{}".format(self.data.iloc[idx].img_path[1:].replace('/', '-'))
        image = transform.resize(io.imread(path), (224, 224))/255
        image = image[:, :, :3]
        
        y = np.where(self.category == self.data.iloc[idx].Category)[0]
        sample = (torch.Tensor(np.einsum('ijk->kij',image)), torch.Tensor(y).long())

        return sample

In [29]:
Category

array(['Coupe', 'Sedan', 'Jeep', 'Hatchback'], dtype=object)

In [30]:
batch_size = 4
train = Dataset_Category('/content/drive/My Drive/Vision/train/category_train.csv')
val = Dataset_Category('/content/drive/My Drive/Vision/test/category_test.csv')

train_loader = DataLoader(train, batch_size=batch_size,shuffle=True, num_workers=4)
val_loader = DataLoader(val, batch_size=batch_size,shuffle=True, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader)}

In [31]:
class model_init(nn.Module):
    def __init__(self):
        super(model_init, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(models.resnet18(pretrained=True))
        self.layers.append(nn.Linear(1000, 512)) 
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(512, 64))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(64, 4))
        self.layers.append(nn.Softmax())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [32]:
def train_helper(category, model, optimization, judge, timeout):
    if category == 'train':
        model.train()
    else:
        model.eval()

    loss_tmp = 0.0
    correct_tmp = 0

    for inputs, labels in dataloaders[category]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0]))
        optimization.zero_grad()

        with torch.set_grad_enabled(category == 'train'):
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = judge(outputs, labels.reshape((-1,)))

            if category == 'train':
                loss.backward()
                optimization.step()

        # statistics
        loss_tmp += loss.item() * inputs.size(0)
        correct_tmp += torch.sum(preds == labels.data)
    if category == 'train':
        timeout.step()

    epoch_loss = loss_tmp / dataset_sizes[category]
    epoch_acc = correct_tmp.double() / dataset_sizes[category]

    print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(category, epoch_loss, epoch_acc/batch_size))
    
    return epoch_acc, copy.deepcopy(model.state_dict())

In [33]:
def train_model(model, judge, optimization, timeout, epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-----------------------------------------')
        train_helper('train', model, optimization, judge, timeout)
        accuracy_tmp, weight_tmp = train_helper('val', model, optimization, judge, timeout)
        if accuracy_tmp > best_accuracy:
            best_accuracy = accuracy_tmp
            best_model_weights = weight_tmp

        print()
        
    print('Best val Acc: {:4f}'.format(best_accuracy))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [34]:
criterion = nn.CrossEntropyLoss()
model = model_init()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, epochs=7)

Epoch 1/7
-----------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


train Loss: 5.0476 Accuracy: 0.4738
val Loss: 5.2519 Accuracy: 0.4300

Epoch 2/7
-----------------------------------------


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/Vision/models/category_model.pt')